In [1]:
import pandas as pd
import numpy as np

from collections import Counter

import re
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


/Users/rutujajangle/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#client_id = 'ee59408d22cb47209e1e8f735beae2d7'
#client_secret = '0bf85ab2cfe74bbf82cf8b565dc3a35b'

client_id = 'db6115dd9210469a8197c3be928129e3'
client_secret = '532606f782c3410e9591071f12f8838a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
auth_url = 'https://accounts.spotify.com/api/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}
auth_response = requests.post(auth_url, data=auth_data)
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
headers = {
    'Authorization': f'Bearer {access_token}',
}

In [4]:
def get_playlist_id_from_url(url):

    playlist_pattern = r'https://open\.spotify\.com/playlist/([a-zA-Z0-9]+)'    
    match = re.search(playlist_pattern, url)

    if match:
        playlist_id = match.group(1)
        return playlist_id
    else:
        return None


def get_playlist_tracks(playlist_id):

    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']

    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    return tracks


def get_track_mood(track_id):

    audio_features = sp.audio_features(track_id)

    if len(audio_features) > 0:
        return audio_features
    
    return None


def get_playlist_features(playlist_tracks):

    song_data = {
        'Track_Name': [],
        'Track_ID': [],
        'URL': [],
        'Artist(s)': [],
        'Album': [],
        'Popularity': [],
        'Duration': [],
        'Tempo': [],
        'Key': [],
        'Mode': [],
        'Danceability': [],
        'Energy': [],
        'Loudness': [],
        'Speechiness': [],
        'Acousticness': [],
        'Instrumentalness': [],
        'Liveliness': [],
        'Valence': [],
        'Time_Signature': []
    }
    
    artist_id = []

    for p in range(len(playlist_tracks)):
        f = get_track_mood(playlist_tracks[p]['track']['uri'])
        song_data['Track_Name'].append(playlist_tracks[p]['track']['name'])
        song_data['Track_ID'].append(playlist_tracks[p]['track']['uri'])
        song_data['URL'].append(playlist_tracks[p]['track']['external_urls']['spotify'])
        song_data['Artist(s)'].append([artist['name'] for artist in playlist_tracks[p]['track']['artists']])
        song_data['Album'].append(playlist_tracks[p]['track']['album']['name'])
        song_data['Popularity'].append(playlist_tracks[p]['track']['popularity'])
        song_data['Duration'].append(round(f[0]['duration_ms'] / 60000, 2))
        song_data['Tempo'].append(f[0]['tempo'])
        song_data['Key'].append(f[0]['key'])
        song_data['Mode'].append(f[0]['mode'])
        song_data['Danceability'].append(f[0]['danceability'])
        song_data['Energy'].append(f[0]['energy'])
        song_data['Loudness'].append(f[0]['loudness'])
        song_data['Speechiness'].append(f[0]['speechiness'])
        song_data['Acousticness'].append(f[0]['acousticness'])
        song_data['Instrumentalness'].append(f[0]['instrumentalness'])
        song_data['Liveliness'].append(f[0]['liveness'])
        song_data['Valence'].append(f[0]['valence'])
        song_data['Time_Signature'].append(f[0]['time_signature'])
        artist_id.extend([artist['id'] for artist in playlist_tracks[p]['track']['artists']])

    artists = {
        'Name': [],
        'ID': [],
        'Genres': [],
        'Followers': [],
        'Popularity': []
    }

    for a in set(artist_id):
        temp = sp.artist('spotify:artist:' + a)
        if temp['genres'] == []:
            continue
        artists['Name'].append(temp['name'])
        artists['ID'].append(temp['id'])
        artists['Genres'].append(temp['genres'])
        artists['Followers'].append(temp['followers']['total'])
        artists['Popularity'].append(temp['popularity'])
    
    return pd.DataFrame(song_data), pd.DataFrame(artists)

In [5]:
def search_songs_by_artist(artist_name):
    
    results = sp.search(q=f'artist:{artist_name}', type='track', limit=1)
    tracks = results['tracks']['items']

    if not tracks:
        return {}

    return tracks


def collab_network(artist_1, artist_2):
    
    collab = {
        "Artist 1": [],
        "Artist 2": [],
        "Popular Collab": [],
        "URL": [], 
        "Popularity": []
    }

    for a in artist_2:
        track = search_songs_by_artist(', '.join([artist_1, a]))
        if track != {}:
            if a in [z['name'] for z in track[0]['artists']] and artist_1 in [z['name'] for z in track[0]['artists']]:
                print(track[0]['name'], track[0]['external_urls']['spotify'], track[0]['popularity'])
                collab["Artist 1"].append(artist_1)
                collab["Artist 2"].append(a)
                collab["Popular Collab"].append(track[0]['name'])
                collab["URL"].append(track[0]['external_urls']['spotify'])
                collab["Popularity"].append(track[0]['popularity'])
    
    return pd.DataFrame(collab)

In [6]:
def get_top_artists_for_genre(genre, num_artists=5):
    results = sp.search(q=f'genre:{genre}', type='artist', limit=num_artists)
    artists = results['artists']['items']
    return artists

def get_top_tracks_for_artist(artist_id, num_tracks=3):
    tracks = sp.artist_top_tracks(artist_id)
    return tracks['tracks'][:num_tracks]

In [7]:
playlist_url_1 = 'https://open.spotify.com/playlist/1EPAHw66Ba1eZEVxffbHgz?si=5fffa0341f7e41eb'
playlist_url_2 = 'https://open.spotify.com/playlist/6GsfHNKqjeScWqakIBQjUM?si=ebf233a66c33468f'
#playlist_url = 'https://open.spotify.com/playlist/1EPAHw66Ba1eZEVxffbHgz?si=369dea96062d476a'
#playlist_url = 'https://open.spotify.com/playlist/3ErWnfK0liTwLCkfXty9KM'
playlist_id_1 = get_playlist_id_from_url(playlist_url_1)
playlist_tracks_1 = get_playlist_tracks(playlist_id_1)
playlist_id_2 = get_playlist_id_from_url(playlist_url_2)
playlist_tracks_2 = get_playlist_tracks(playlist_id_2)

In [8]:
playlist1, artist1 = get_playlist_features(playlist_tracks_1)
playlist2, artist2 = get_playlist_features(playlist_tracks_2)

In [9]:
playlist1['Playlist'] = 'Playlist 1'
playlist2['Playlist'] = 'Playlist 2'

In [10]:
playlists = pd.concat([playlist1, playlist2], ignore_index=True)
playlists.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Playlist
3,High Hopes,spotify:track:6KkyuDhrEhR5nJVKtv9mCf,https://open.spotify.com/track/6KkyuDhrEhR5nJV...,[Kodaline],In A Perfect World (Expanded Edition),62,3.84,77.278,4,1,0.488,0.487,-6.371,0.0305,0.5770,0.00000,0.1930,0.219,4,Playlist 1
16,Wake Me up When September Ends,spotify:track:3ZffCQKLFLUvYM59XKLbVm,https://open.spotify.com/track/3ZffCQKLFLUvYM5...,[Green Day],American Idiot,77,4.76,104.980,7,1,0.546,0.814,-5.146,0.0314,0.0230,0.00000,0.0972,0.146,4,Playlist 1
97,Middle,spotify:track:0g5EKLgdKvNlln7TNqBByK,https://open.spotify.com/track/0g5EKLgdKvNlln7...,"[DJ Snake, Bipolar Sunshine]",Encore,73,3.68,104.879,9,1,0.583,0.695,-5.336,0.0423,0.0138,0.00000,0.0527,0.224,4,Playlist 2
62,Our Song,spotify:track:3fGW43fnu3OHEw6ohTC2Bw,https://open.spotify.com/track/3fGW43fnu3OHEw6...,"[Anne-Marie, Niall Horan]",Therapy,61,2.73,80.950,2,1,0.572,0.759,-4.312,0.0506,0.1270,0.00000,0.2240,0.519,4,Playlist 2
39,When the Day Met the Night,spotify:track:64TQPdiQqWknjFhPL9QmxJ,https://open.spotify.com/track/64TQPdiQqWknjFh...,[Panic! At The Disco],Pretty. Odd.,42,4.90,130.276,7,1,0.432,0.656,-6.889,0.0419,0.0618,0.00006,0.3910,0.171,4,Playlist 1


In [11]:
all_artists = pd.concat([artist1, artist2], ignore_index=True)
which_playlist = []
for a in all_artists['Name']:
    if a in list(artist1['Name']) and a in list(artist2['Name']):
        which_playlist.append("Both")
    elif a in list(artist1['Name']):
        which_playlist.append("Playlist 1")
    else:
        which_playlist.append("Playlist 2")
all_artists['Playlist'] = which_playlist
all_artists = all_artists.loc[all_artists.astype(str).drop_duplicates().index]
all_artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist
48,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,"[canadian pop, pop]",78768494,91,Playlist 2
51,Bastille,7EQ0qTo7fWT7DPxmxtSYEc,"[metropopolis, modern rock, pop]",5585188,76,Playlist 2
39,Lizzo,56oDRnqbIiwx4mymNEv7dS,"[escape room, minnesota hip hop, pop, trap queen]",5713738,70,Playlist 2
6,Eagles,0ECwFtbIWEVNwjlrfc6xoL,"[album rock, classic rock, heartland rock, mel...",11155447,77,Playlist 1
20,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,"[alternative metal, nu metal, post-grunge, rap...",27616056,93,Playlist 1


In [12]:
a1 = dict(Counter(list(playlist1.explode('Artist(s)')['Artist(s)'])))
a2 = dict(Counter(list(playlist2.explode('Artist(s)')['Artist(s)'])))

In [13]:
tc = []
p1 = []
p2 = []
for a in list(all_artists['Name']):
    c1 = 0
    c2 = 0
    if a in list(a1.keys()):
        c1 = a1[a]
    if a in list(a2.keys()):
        c2 = a2[a]
    tc.append(c1+c2)
    p1.append(c1)
    p2.append(c2)

all_artists['Total Count'] = tc
all_artists['Count in Playlist 1'] = p1
all_artists['Count in Playlist 2'] = p2
all_artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2
26,Maroon 5,04gDigrS5kc9YWfZHwBETP,[pop],43205328,87,Both,3,1,2
86,Zedd,2qxJFvFYMEDqd7ui6kSAcq,"[complextro, edm, german techno, pop, pop dance]",5986151,76,Playlist 2,1,0,1
3,Snow Patrol,3rIZMv9rysU7JkLzEaC5Jp,"[irish rock, modern rock, neo mellow, permanen...",3094812,71,Playlist 1,1,1,0
81,Tainy,0GM7qgcRCORpGnfcN2tCiB,"[pop reggaeton, reggaeton, trap latino, urbano...",2473636,79,Playlist 2,1,0,1
44,Nicky Youre,7qmpXeNz2ojlMl2EEfkeLs,[singer-songwriter pop],278391,70,Playlist 2,1,0,1


In [14]:
artist_presence = {}
for a, p in zip(list(all_artists['Name']), list(all_artists['Playlist'])):
    artist_presence[a] = p

In [15]:
artist_list = np.array(all_artists['Name'])
x = 0
lim = len(artist_list) // 10
y = x + lim
artist_dict = {}
print(x, y, lim)

0 8 8


In [16]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    x = y
    y = len(artist_list)
print(x, y, lim)

Automatic Yes (feat. John Mayer) https://open.spotify.com/track/6oAwQp0fXsZObpl5PLl2GN 55
Magnolia (feat. John Mayer) https://open.spotify.com/track/0VzLAYIns618H651sG09Gp 50
This Is The Time (feat. John Mayer) - Live at Shea Stadium, Queens, NY - July 2008 https://open.spotify.com/track/5Wf8sdQL7e9vHOVBDdGRdd 26
C'mon - Single Version https://open.spotify.com/track/53c9tTEHXWL5GkYNalbXZ3 50
Runaway Horses (feat. Phoebe Bridgers) - Abridged https://open.spotify.com/track/3PeCWDczrGqdD1E3mbok2X 41
Caution - Clean Bandit Remix https://open.spotify.com/track/73A4FC5eIihO0swjveMDTl 28
On/Off https://open.spotify.com/track/3JNGLHIM47zhjcJZJ5DTY2 2
My Universe - David Guetta Remix https://open.spotify.com/track/5FvxRvrQ8qzKjBJ6ST9aiu 45
My Universe https://open.spotify.com/track/46HNZY1i7O6jwTA7Slo2PI 75
Let Somebody Go https://open.spotify.com/track/4cGqn0E8JCSY9gQllQj4Mf 66
Something Just Like This https://open.spotify.com/track/6RUKPb4LETWmmr3iAEQktW 86
Midnight - Kygo Remix https://open.

In [17]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Raging (feat. Kodaline) https://open.spotify.com/track/2Vhd5uZB4brKHmM0MNxkZo 59
Pistol of Fire - Mark Ronson Remix https://open.spotify.com/track/2qOOsAgIpqXElQN7fkfj9N 18
Goodbye Friend (feat. The Script) https://open.spotify.com/track/59naFBAHqgOUBzTTSCjYOK 34
16 24 8


In [18]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

This Is The Time (feat. John Mayer) - Live at Shea Stadium, Queens, NY - July 2008 https://open.spotify.com/track/5Wf8sdQL7e9vHOVBDdGRdd 26
Runaway Horses (feat. Phoebe Bridgers) - Abridged https://open.spotify.com/track/3PeCWDczrGqdD1E3mbok2X 41
Nothing New (feat. Phoebe Bridgers) (Taylor’s Version) (From The Vault) https://open.spotify.com/track/01K4zKU104LyJ8gMb7227B 71
I Don't Wanna Wait https://open.spotify.com/track/331l3xABO0HMr1Kkyh2LZq 85
Lose Somebody https://open.spotify.com/track/1sgDyuLooyvEML4oHspNza 64
The Last Time (feat. Gary Lightbody of Snow Patrol) (Taylor’s Version) https://open.spotify.com/track/0y6kdSRCVQhSsHSpWvTUm7 67
24 32 8


In [19]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Button (feat. Anuel AA & Tainy) https://open.spotify.com/track/6FDKYM0lBf6zqrg8cFokPd 44
Payphone https://open.spotify.com/track/4P0osvTXoSYZZC2n8IFH3c 70
Beautiful Mistakes (feat. Megan Thee Stallion) https://open.spotify.com/track/5zFglKYiknIxks8geR8rcL 74
ME! (feat. Brendon Urie of Panic! At The Disco) https://open.spotify.com/track/2Rk4JlNc2TPmZe2af99d45 72
Give Me One Reason - Live https://open.spotify.com/track/6jTqomGlRFQ52EgIIWBfnY 39
Magnolia (feat. John Mayer) https://open.spotify.com/track/0VzLAYIns618H651sG09Gp 50
Atomic City - David Guetta Remix https://open.spotify.com/track/6pXZHDf8k1Zhv4khk4spIx 34
The Saints Are Coming https://open.spotify.com/track/6uQL0epVs9N9Bergafaf6x 37
You’re The Best Thing About Me - U2 Vs. Kygo https://open.spotify.com/track/4QWSk7Ynu34r3EOHSwp9nI 40
Give Me One Reason - Live https://open.spotify.com/track/6jTqomGlRFQ52EgIIWBfnY 39
32 40 8


In [20]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

BLOW (with Chris Stapleton & Bruno Mars) https://open.spotify.com/track/6EbduYTr62sSzvl36wWiFM 61
I Don't Care (with Justin Bieber) https://open.spotify.com/track/0hVXuCcriWRGvwMV1r5Yn9 78
Dark Times https://open.spotify.com/track/6J7PIkhtHwmFjPLq0SyuPj 58
South of the Border (feat. Camila Cabello & Cardi B) https://open.spotify.com/track/4vUmTMuQqjdnvlZmAH61Qk 70
End Game https://open.spotify.com/track/2x0WlnmfG39ZuDmstl9xfX 72
The Bones - with Hozier https://open.spotify.com/track/1yTTMcUhL7rtz08Dsgb7Qb 61
End Game https://open.spotify.com/track/2x0WlnmfG39ZuDmstl9xfX 72
You All Over Me (feat. Maren Morris) (Taylor’s Version) (From The Vault) https://open.spotify.com/track/4CHpVfAhuxNJ3ibExe6kxO 57
Lover (Remix) [feat. Shawn Mendes] https://open.spotify.com/track/3i9UVldZOE0aD0JnyfAZZ0 65
Nothing New (feat. Phoebe Bridgers) (Taylor’s Version) (From The Vault) https://open.spotify.com/track/01K4zKU104LyJ8gMb7227B 71
You Need To Calm Down - Clean Bandit Remix https://open.spotify.com/t

In [21]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

I Don’t Wanna Live Forever (Fifty Shades Darker) https://open.spotify.com/track/55n9yjI6qqXh5F2mYvUc2y 79
Dusk Till Dawn (feat. Sia) - Radio Edit https://open.spotify.com/track/3e7sxremeOE3wTySiOhGiP 80
Seeing Blind https://open.spotify.com/track/65wnZsZLKJimfK1cTFOfa1 59
Our Song https://open.spotify.com/track/3fGW43fnu3OHEw6ohTC2Bw 61
Cry Baby https://open.spotify.com/track/7oDSG2Q9RBU0QjgtfYQOdD 73
Caution - Clean Bandit Remix https://open.spotify.com/track/73A4FC5eIihO0swjveMDTl 28
You Need To Calm Down - Clean Bandit Remix https://open.spotify.com/track/7Bjf96uNizpvXLvNyOJpst 44
Symphony (feat. Zara Larsson) https://open.spotify.com/track/72gv4zhNvRVdQA0eOenCal 77
Rockabye (feat. Sean Paul & Anne-Marie) https://open.spotify.com/track/2hrUO4drrO63i7FYbCLBl2 74
Stop Crying Your Heart Out - BBC Radio 2 Allstars https://open.spotify.com/track/0vJN0jvhJ5nknlcIdDdlD3 34
New Eyes (feat. Lizzo) https://open.spotify.com/track/6xyJHp3nAJGOc2Xy7ACsg1 21
Alone - Clean Bandit MFF Remix https:/

In [22]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Starboy (feat. Daft Punk) - Kygo Remix https://open.spotify.com/track/3DCc4HPrVukBeaDGe3Cosk 48
Often - Kygo Remix https://open.spotify.com/track/27NV2KxoQ8WuLMqlTDI61F 61
Like It Is https://open.spotify.com/track/3frUvGrmGcay91lvFbOgsN 54
It Ain’t Me (with Selena Gomez) https://open.spotify.com/track/12GEpg2XOPyqk03JZEZnJs 64
Midnight - Kygo Remix https://open.spotify.com/track/2CoWJhDYVENsByS7rPInSr 46
Forever Yours - Avicii Tribute https://open.spotify.com/track/3o1CUVeHIid49sabk6A6Nf 58
Family https://open.spotify.com/track/3TKpJrY9q49Mj1JOsM9zGL 54
Raging (feat. Kodaline) https://open.spotify.com/track/2Vhd5uZB4brKHmM0MNxkZo 59
You’re The Best Thing About Me - U2 Vs. Kygo https://open.spotify.com/track/4QWSk7Ynu34r3EOHSwp9nI 40
Lose Somebody https://open.spotify.com/track/1sgDyuLooyvEML4oHspNza 64
Torn Apart, Pt II (Bastille Vs. GRADES Vs. Lizzo) https://open.spotify.com/track/6Gy30w4ebJg02BjLdULdNm 27
Happier https://open.spotify.com/track/7BqHUALzNBTanL6OvsqmC1 82
Uptown Funk (f

In [23]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Where Are Ü Now (with Justin Bieber) - Marshmello Remix https://open.spotify.com/track/5ScMaEiUXf9BATTbDKNFq8 42
Happier https://open.spotify.com/track/7BqHUALzNBTanL6OvsqmC1 82
Wolves https://open.spotify.com/track/0tBbt8CrmxbjRP0pueQkyU 75
FRIENDS https://open.spotify.com/track/08bNPGLD8AhKpnnERrAc6G 73
Waiting For Love - Marshmello Remix https://open.spotify.com/track/4gnvnnm2oYy8IsUZ104wOS 32
Hate The Other Side (with Marshmello & The Kid Laroi) https://open.spotify.com/track/0ada5XsQGLPUVbmTCkAP49 68
Be Kind (with Halsey) https://open.spotify.com/track/3Z8FwOEN59mRMxDCtb8N0A 70
Let Me Love You - Marshmello Remix https://open.spotify.com/track/6Hx9gLuAKkOfoZoL79OSI1 45
Bad Bitches (feat. Megan Thee Stallion) https://open.spotify.com/track/2OYfxjLnQuPo7Co1WH0Z1S 36
Sweetest Pie - David Guetta Dance Remix https://open.spotify.com/track/5xUJYQA7f6WU77N8VDASBH 42
Electricity https://open.spotify.com/track/5N4erncE7kuUccm7zEmwzk 66
Prisoner (feat. Dua Lipa) https://open.spotify.com/trac

In [24]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

The Feeling https://open.spotify.com/track/7E390nZTMqEbrNC1TmHd42 57
Boy With Luv (feat. Halsey) https://open.spotify.com/track/4a9tbd947vo9K8Vti9JwcI 73
Alone - Clean Bandit MFF Remix https://open.spotify.com/track/483v27PMHWBtPFyXQ3rElQ 25
Closer https://open.spotify.com/track/7BKLCZ1jbUBVqRi2FVlTVw 84
Be Kind (with Halsey) https://open.spotify.com/track/3Z8FwOEN59mRMxDCtb8N0A 70
So Far Away (feat. Jamie Scott & Romy Dya) https://open.spotify.com/track/0OlnLZY4cmQzT6ZGttvWBM 62
Can't Feel My Face - Martin Garrix Remix https://open.spotify.com/track/22SXyL6pUjwIvGWvgGjLov 48
Follow https://open.spotify.com/track/5aXTfpNOqLj35ydEiLio67 52
Scared to Be Lonely https://open.spotify.com/track/3ebXMykcMXOcLeJ9xZ17XH 78
Titanium (feat. Sia) https://open.spotify.com/track/0lHAMNU8RGiIObScrsRgmP 69
ON (Feat. Sia) https://open.spotify.com/track/4JWbH50WabthGhmtduR8b9 52
Elastic Heart - From "The Hunger Games: Catching Fire" Soundtrack https://open.spotify.com/track/6x0ex5urwhqB3AYLREVRUd 53
Dus

In [25]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

The Bones - with Hozier https://open.spotify.com/track/1yTTMcUhL7rtz08Dsgb7Qb 61
You All Over Me (feat. Maren Morris) (Taylor’s Version) (From The Vault) https://open.spotify.com/track/4CHpVfAhuxNJ3ibExe6kxO 57
The Middle https://open.spotify.com/track/09IStsImFySgyp0pIQdqAc 76
The Middle https://open.spotify.com/track/09IStsImFySgyp0pIQdqAc 76
Seeing Blind https://open.spotify.com/track/65wnZsZLKJimfK1cTFOfa1 59
Baby Don't Hurt Me https://open.spotify.com/track/3BKD1PwArikchz2Zrlp1qi 81
Rockabye (feat. Sean Paul & Anne-Marie) https://open.spotify.com/track/2hrUO4drrO63i7FYbCLBl2 74
fuck, i'm lonely https://open.spotify.com/track/2RNsweBBb1a6tAHXQtt2xH 63
Our Song https://open.spotify.com/track/3fGW43fnu3OHEw6ohTC2Bw 61
FRIENDS https://open.spotify.com/track/08bNPGLD8AhKpnnERrAc6G 73
Let Me Love You https://open.spotify.com/track/0lYBSQXN6rCTvUZvg9S0lU 83
Let Me Love You - Zedd Remix https://open.spotify.com/track/7xufmkFUZoSjX1mxWupnKY 34
Taki Taki (feat. Selena Gomez, Ozuna & Cardi B

In [26]:
for i in range(x, y):
    artist_1 = artist_list[i]
    artist_2 = [z for z in set(artist_list).difference({artist_1})]
    artist_dict[artist_1] = collab_network(artist_1, artist_2)
x += lim
y += lim
if y > len(artist_list):
    y = len(artist_list)
print(x, y, lim)

Let Me Love You - Zedd Remix https://open.spotify.com/track/7xufmkFUZoSjX1mxWupnKY 34
The Middle https://open.spotify.com/track/09IStsImFySgyp0pIQdqAc 76
Lost In Japan - Remix https://open.spotify.com/track/575NJxNUVDqwJGdzBrlLbv 55
I Want You To Know https://open.spotify.com/track/1JDIArrcepzWDTAWXdGYmP 63
Marry The Night - Zedd Remix https://open.spotify.com/track/3acHQsIoaXtCPX96CDgivm 25
The Middle https://open.spotify.com/track/09IStsImFySgyp0pIQdqAc 76
Follow https://open.spotify.com/track/5aXTfpNOqLj35ydEiLio67 52
Let Me Love You - Zedd Remix https://open.spotify.com/track/7xufmkFUZoSjX1mxWupnKY 34
Break Free https://open.spotify.com/track/12KUFSHFgT0XCoiSlvdQi4 75
Automatic Yes (feat. John Mayer) https://open.spotify.com/track/6oAwQp0fXsZObpl5PLl2GN 55
Up https://open.spotify.com/track/2IWTYdKpX1cpm3Ogefn9kb 22
Die With A Smile https://open.spotify.com/track/2plbrEY59IikOBgBGLjaoe 100
Marry The Night - The Weeknd & Illangelo Remix https://open.spotify.com/track/5bfivmKwgSni611w

In [27]:
collabs = pd.concat(artist_dict.values(), ignore_index=True)

In [28]:
presence = []
for a1, a2 in zip(list(collabs['Artist 1']), list(collabs['Artist 2'])):
    if artist_presence[a1] == artist_presence[a2] and artist_presence[a1] != "Both":
        presence.append("Artists from Same Playlist")
    if artist_presence[a1] == artist_presence[a2] and artist_presence[a1] == "Both":
        presence.append("Both Artists in Same Playlists")
    if artist_presence[a1] != artist_presence[a2] :
        presence.append("Artists in Different Playlists")

collabs['Artist From'] = presence
collabs.sample(5)

,Artist 1,Artist 2,Popular Collab,URL,Popularity,Artist From
236,Tainy,Camila Cabello,Oh Na Na,https://open.spotify.com/track/03NqHNdG3Ur6kci...,50.0,Artists from Same Playlist
237,Tainy,Maroon 5,Button (feat. Anuel AA & Tainy),https://open.spotify.com/track/6FDKYM0lBf6zqrg...,44.0,Artists in Different Playlists
144,Marshmello,Anne-Marie,FRIENDS,https://open.spotify.com/track/08bNPGLD8AhKpnn...,73.0,Artists from Same Playlist
73,Wiz Khalifa,Bruno Mars,"Young, Wild & Free (feat. Bruno Mars)",https://open.spotify.com/track/5HQVUIKwCEXpe7J...,74.0,Artists from Same Playlist
226,DJ Snake,Selena Gomez,"Taki Taki (feat. Selena Gomez, Ozuna & Cardi B)",https://open.spotify.com/track/0KoiMHhqKLoL4xM...,71.0,Artists from Same Playlist


In [29]:
genre1 = dict(Counter(list(artist1.explode(column='Genres')['Genres'])))
#genre1 = {'Genre': list(genre1.keys()),
#          'Count': list(genre1.values())}
genre2 = dict(Counter(list(artist2.explode(column='Genres')['Genres'])))
#genre2 = {'Genre': list(genre2.keys()),
#          'Count': list(genre2.values())}

In [30]:
genre_list = []
genre_list.extend(list(genre1.keys()))
genre_list.extend(list(genre2.keys()))

genre_list = [x for x in set(genre_list)]

In [31]:
tc = []
cp1 = []
cp2 = []
pl = []

for g in genre_list:
    p1 = 0
    p2 = 0
    f = 0
    if g in genre1.keys():
        p1 = genre1[g]
    if g in genre2.keys():
        p2 = genre2[g]
    tc.append(p1+p2)
    cp1.append(p1)
    cp2.append(p2)
    if p1 == 0 and p2 != 0:
        pl.append("Playlist 2")
    elif p1 != 0 and p2 == 0:
        pl.append("Playlist 1")
    else:
        pl.append("Both")

all_genres = pd.DataFrame({
    'Genre': genre_list,
    'Playlist': pl,
    'Total Count': tc,
    'Count in Playlist 1': cp1,
    'Count in Playlist 2': cp2
})

all_genres.sample(5)

,Genre,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2
89,nu metal,Playlist 1,1,1,0
104,uk americana,Playlist 1,1,1,0
88,swedish pop,Both,2,1,1
102,german techno,Playlist 2,1,0,1
68,electropop,Playlist 2,1,0,1


In [32]:
cg = list(all_genres[all_genres['Playlist'] == "Both"]['Genre'])
common_genres = []
for g in cg:
    g = g.split()[0]
    common_genres.append(g)

common_artists = list(all_artists[all_artists['Playlist'] == "Both"]['Name'])
common_artists_id = list(all_artists[all_artists['Playlist'] == "Both"]['ID'])

In [33]:
common_genre_top_artists = pd.DataFrame()
for g1, g2 in zip(common_genres, cg):
    an = []
    aid = []
    ag = []
    ap = []
    #print(g2)
    genre_artists = get_top_artists_for_genre(g1, 25)
    for a in genre_artists:
        gc = a['genres']
        if g2 in gc and a['name'] not in common_artists:
            an.append(a['name'])
            aid.append(a['id'])
            ap.append(a['popularity'])
            ag.append(g2)
    ga = pd.DataFrame({
        'Name': an,
        'ID': aid,
        'Genre': ag,
        'Popularity': ap
    }).sort_values(by='Popularity', ascending=False, ignore_index=True)
    if len(ga) > 5:
        ga = ga.head(5)
    #print(ga)
    common_genre_top_artists = pd.concat([common_genre_top_artists, ga])
    #print()
common_genre_top_artists.sample(5)

,Name,ID,Genre,Popularity
1,Empire Of The Sun,67hb7towEyKvt5Z8Bx306c,indietronica,78.0
4,Bastille,7EQ0qTo7fWT7DPxmxtSYEc,metropopolis,76.0
2,TV Girl,0Y6dVaC9DZtPNH4591M42W,pov: indie,84.0
4,Disclosure,6nS5roXSAGhTGr34W6n7Et,indietronica,76.0
1,Vance Joy,10exVja0key0uqUkk6LJRT,folk-pop,78.0


In [34]:
scg = []
songs_common_genres = pd.DataFrame()
for a, g, i in zip(list(common_genre_top_artists['Name']), list(common_genre_top_artists['Genre']), list(common_genre_top_artists['ID'])):
    #print(a)
    common_genre_songs = {
        'Track_Name': [],
        'Track_ID': [],
        'URL': [],
        'Artist(s)': [],
        'Album': [],
        'Popularity': [],
        'Duration': [],
        'Tempo': [],
        'Key': [],
        'Mode': [],
        'Danceability': [],
        'Energy': [],
        'Loudness': [],
        'Speechiness': [],
        'Acousticness': [],
        'Instrumentalness': [],
        'Liveliness': [],
        'Valence': [],
        'Time_Signature': []
    }
    songs = get_top_tracks_for_artist(i, num_tracks=25)
    for s in songs:
        #print(s['name'], [artist['name'] for artist in s['artists']])
        if s['uri'] not in list(playlists['Track_ID']):
            #print(s['name'], [artist['name'] for artist in s['artists']])
            #print()
            f = get_track_mood(s['uri'])
            common_genre_songs['Track_Name'].append(s['name'])
            common_genre_songs['Track_ID'].append(s['uri'])
            common_genre_songs['URL'].append(s['external_urls']['spotify'])
            common_genre_songs['Artist(s)'].append([artist['name'] for artist in s['artists']])
            common_genre_songs['Album'].append(s['album']['name'])
            common_genre_songs['Popularity'].append(s['popularity'])
            common_genre_songs['Duration'].append(round(f[0]['duration_ms'] / 60000, 2))
            common_genre_songs['Tempo'].append(f[0]['tempo'])
            common_genre_songs['Key'].append(f[0]['key'])
            common_genre_songs['Mode'].append(f[0]['mode'])
            common_genre_songs['Danceability'].append(f[0]['danceability'])
            common_genre_songs['Energy'].append(f[0]['energy'])
            common_genre_songs['Loudness'].append(f[0]['loudness'])
            common_genre_songs['Speechiness'].append(f[0]['speechiness'])
            common_genre_songs['Acousticness'].append(f[0]['acousticness'])
            common_genre_songs['Instrumentalness'].append(f[0]['instrumentalness'])
            common_genre_songs['Liveliness'].append(f[0]['liveness'])
            common_genre_songs['Valence'].append(f[0]['valence'])
            common_genre_songs['Time_Signature'].append(f[0]['time_signature'])
    common_genre_songs = pd.DataFrame(common_genre_songs).sort_values(by='Popularity', ascending=False, ignore_index=True).head(5)
    #print(common_genre_songs)
    #print()
    songs_common_genres = pd.concat([songs_common_genres, common_genre_songs], ignore_index=True)
    scg.extend([g]*len(common_genre_songs))
songs_common_genres.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature
161,Everybody's Changing,spotify:track:5hkdfA87RZvNaxl6XiveOA,https://open.spotify.com/track/5hkdfA87RZvNaxl...,[Keane],Hopes And Fears,64,3.59,94.020,0,1,0.517,0.832,-3.564,0.0292,0.0015,0.000208,0.1010,0.450,4
133,Gravity,spotify:track:3SktMqZmo3M9zbB7oKMIF7,https://open.spotify.com/track/3SktMqZmo3M9zbB...,[John Mayer],Continuum,72,4.10,123.799,7,1,0.753,0.326,-9.883,0.0342,0.7310,0.001310,0.2400,0.237,3
219,Juno,spotify:track:21B4gaTWnTkuSh77iWEXdS,https://open.spotify.com/track/21B4gaTWnTkuSh7...,[Sabrina Carpenter],Short n' Sweet,87,3.72,130.504,5,1,0.633,0.725,-4.257,0.0427,0.0461,0.000000,0.2050,0.640,4
56,The Thrill,spotify:track:56oGoEjA9eTZYgsttEFKY3,https://open.spotify.com/track/56oGoEjA9eTZYgs...,"[Wiz Khalifa, Empire Of The Sun]",The Thrill (feat. Empire of the Sun),76,3.91,126.984,10,0,0.817,0.775,-4.767,0.1080,0.2770,0.000000,0.1080,0.806,4
13,Georgia,spotify:track:5sfvzgf96i6nLyLELnrLZ6,https://open.spotify.com/track/5sfvzgf96i6nLyL...,[Vance Joy],Dream Your Life Away (Deluxe Edition),67,3.84,143.554,11,1,0.388,0.659,-7.372,0.0308,0.3120,0.000315,0.0943,0.401,4


In [35]:
ca = []
songs_common_artists = pd.DataFrame()
for a, i in zip(common_artists, common_artists_id):
    common_artist_songs = {
        'Track_Name': [],
        'Track_ID': [],
        'URL': [],
        'Artist(s)': [],
        'Album': [],
        'Popularity': [],
        'Duration': [],
        'Tempo': [],
        'Key': [],
        'Mode': [],
        'Danceability': [],
        'Energy': [],
        'Loudness': [],
        'Speechiness': [],
        'Acousticness': [],
        'Instrumentalness': [],
        'Liveliness': [],
        'Valence': [],
        'Time_Signature': []
    }
    songs = get_top_tracks_for_artist(i, num_tracks=25)
    for s in songs:
        if s['uri'] not in list(playlists['Track_ID']):
            f = get_track_mood(s['uri'])
            common_artist_songs['Track_Name'].append(s['name'])
            common_artist_songs['Track_ID'].append(s['uri'])
            common_artist_songs['URL'].append(s['external_urls']['spotify'])
            common_artist_songs['Artist(s)'].append([artist['name'] for artist in s['artists']])
            common_artist_songs['Album'].append(s['album']['name'])
            common_artist_songs['Popularity'].append(s['popularity'])
            common_artist_songs['Duration'].append(round(f[0]['duration_ms'] / 60000, 2))
            common_artist_songs['Tempo'].append(f[0]['tempo'])
            common_artist_songs['Key'].append(f[0]['key'])
            common_artist_songs['Mode'].append(f[0]['mode'])
            common_artist_songs['Danceability'].append(f[0]['danceability'])
            common_artist_songs['Energy'].append(f[0]['energy'])
            common_artist_songs['Loudness'].append(f[0]['loudness'])
            common_artist_songs['Speechiness'].append(f[0]['speechiness'])
            common_artist_songs['Acousticness'].append(f[0]['acousticness'])
            common_artist_songs['Instrumentalness'].append(f[0]['instrumentalness'])
            common_artist_songs['Liveliness'].append(f[0]['liveness'])
            common_artist_songs['Valence'].append(f[0]['valence'])
            common_artist_songs['Time_Signature'].append(f[0]['time_signature'])
    common_artist_songs = pd.DataFrame(common_artist_songs).sort_values(by='Popularity', ascending=False, ignore_index=True).head(5)
    songs_common_artists = pd.concat([songs_common_artists, common_artist_songs], ignore_index=True)
    ca.extend([a]*len(common_artist_songs))
songs_common_artists.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature
28,Bad Habits,spotify:track:3rmo8F54jFF8OgYsqTxm5d,https://open.spotify.com/track/3rmo8F54jFF8OgY...,[Ed Sheeran],=,79,3.85,126.011,11,0,0.807,0.893,-3.745,0.0347,0.045100,0.000028,0.3660,0.537,4
20,House of Memories,spotify:track:2DgdHcjWmO3qd50RzuBLgZ,https://open.spotify.com/track/2DgdHcjWmO3qd50...,[Panic! At The Disco],Death of a Bachelor,77,3.48,109.952,11,0,0.506,0.817,-5.051,0.0335,0.002720,0.000000,0.0486,0.506,4
5,Friday I'm in Love,spotify:track:263aNAQCeFSWipk896byo6,https://open.spotify.com/track/263aNAQCeFSWipk...,[The Cure],Wish,68,3.57,136.203,0,0,0.526,0.762,-12.437,0.0342,0.000972,0.000116,0.3380,0.505,4
22,I Write Sins Not Tragedies,spotify:track:4bPQs0PHn4xbipzdPfn6du,https://open.spotify.com/track/4bPQs0PHn4xbipz...,[Panic! At The Disco],A Fever You Can't Sweat Out,71,3.11,170.094,9,0,0.569,0.830,-4.106,0.1400,0.080700,0.000000,0.1140,0.609,4
31,Lover,spotify:track:1dGr1c8CrMLDpV6mPbImSI,https://open.spotify.com/track/1dGr1c8CrMLDpV6...,[Taylor Swift],Lover,84,3.69,68.534,7,1,0.359,0.543,-7.582,0.0919,0.492000,0.000016,0.1180,0.453,4


In [36]:
new_playlist = pd.concat([songs_common_genres, songs_common_artists], ignore_index=True).sort_values(by='Tempo', ignore_index=True)
new_playlist.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature
135,Shake It Out,spotify:track:4lY95OMGb9WxP6IYut64ir,https://open.spotify.com/track/4lY95OMGb9WxP6I...,[Florence + The Machine],Ceremonials (Deluxe Edition),62,4.63,107.480,10,1,0.528,0.804,-4.168,0.0358,0.0104,0.000000,0.1050,0.222,4
271,Just Can't Get Enough,spotify:track:0qi4b1l0eT3jpzeNHeFXDT,https://open.spotify.com/track/0qi4b1l0eT3jpze...,[Depeche Mode],Speak and Spell (Deluxe),74,3.68,128.168,0,1,0.766,0.772,-9.362,0.0353,0.4030,0.058400,0.1830,0.919,4
90,Shape of You,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://open.spotify.com/track/7qiZfU4dY1lWllz...,[Ed Sheeran],÷ (Deluxe),86,3.90,95.977,1,0,0.825,0.652,-3.183,0.0802,0.5810,0.000000,0.0931,0.931,4
50,High and Dry,spotify:track:2a1iMaoWQ5MnvLFBDv4qkf,https://open.spotify.com/track/2a1iMaoWQ5MnvLF...,[Radiohead],The Bends,75,4.29,87.568,4,1,0.419,0.383,-11.782,0.0256,0.0724,0.017600,0.0896,0.350,4
72,Why'd You Only Call Me When You're High?,spotify:track:086myS9r57YsLbJpU0TgK9,https://open.spotify.com/track/086myS9r57YsLbJ...,[Arctic Monkeys],AM,84,2.69,92.004,2,1,0.691,0.631,-6.478,0.0368,0.0483,0.000011,0.1040,0.800,4


In [37]:
songs_common_genres['Common Genre'] = scg
songs_common_genres.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Genre
62,The Other Side Of Paradise,spotify:track:0rRjGruFonCGOt0S5zAJNQ,https://open.spotify.com/track/0rRjGruFonCGOt0...,[Glass Animals],How To Be A Human Being,67,5.34,128.012,11,0,0.689,0.338,-7.634,0.0629,0.34900,0.000057,0.0649,0.568,4,indietronica
147,Drive By,spotify:track:0KAiuUOrLTIkzkpfpn9jb9,https://open.spotify.com/track/0KAiuUOrLTIkzkp...,[Train],California 37,79,3.27,122.028,1,0,0.765,0.837,-3.113,0.0320,0.00107,0.000011,0.0801,0.721,4,neo mellow
86,Why'd You Only Call Me When You're High?,spotify:track:086myS9r57YsLbJpU0TgK9,https://open.spotify.com/track/086myS9r57YsLbJ...,[Arctic Monkeys],AM,84,2.69,92.004,2,1,0.691,0.631,-6.478,0.0368,0.04830,0.000011,0.1040,0.800,4,rock
41,Easy On Me,spotify:track:46IZ0fSY2mpAiktS3KOqds,https://open.spotify.com/track/46IZ0fSY2mpAikt...,[Adele],30,82,3.74,141.981,5,1,0.604,0.366,-7.519,0.0282,0.57800,0.000000,0.1330,0.130,4,uk pop
109,Ghost Town,spotify:track:6HuHyXviAdpmnQu73JHfBz,https://open.spotify.com/track/6HuHyXviAdpmnQu...,[Benson Boone],Fireworks & Rollerblades,76,3.23,174.185,0,1,0.391,0.335,-10.064,0.0763,0.38900,0.000000,0.1020,0.116,3,singer-songwriter pop


In [38]:
songs_common_artists['Common Artist'] = ca
songs_common_artists.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Artist
19,Sugar,spotify:track:2iuZJX9X9P0GKaE93xcPjk,https://open.spotify.com/track/2iuZJX9X9P0GKaE...,[Maroon 5],V,78,3.92,120.076,1,1,0.748,0.788,-7.055,0.0334,0.05910,0.000000,0.0863,0.884,4,Maroon 5
16,Memories,spotify:track:4cktbXiXOapiLBMprHFErI,https://open.spotify.com/track/4cktbXiXOapiLBM...,[Maroon 5],JORDI (Deluxe),82,3.16,91.050,11,1,0.775,0.327,-7.241,0.0557,0.84100,0.000000,0.0821,0.595,4,Maroon 5
0,Yellow,spotify:track:3AJwUDP919kvQ9QcozQPxg,https://open.spotify.com/track/3AJwUDP919kvQ9Q...,[Coldplay],Parachutes,89,4.45,173.372,11,1,0.429,0.661,-7.227,0.0281,0.00239,0.000121,0.2340,0.285,4,Coldplay
32,I Can Do It With a Broken Heart,spotify:track:4q5YezDOIPcoLr8R81x9qy,https://open.spotify.com/track/4q5YezDOIPcoLr8...,[Taylor Swift],THE TORTURED POETS DEPARTMENT,83,3.63,129.994,0,1,0.701,0.751,-8.390,0.0502,0.12300,0.000000,0.1500,0.220,4,Taylor Swift
21,ME! (feat. Brendon Urie of Panic! At The Disco),spotify:track:2Rk4JlNc2TPmZe2af99d45,https://open.spotify.com/track/2Rk4JlNc2TPmZe2...,"[Taylor Swift, Brendon Urie, Panic! At The Disco]",Lover,72,3.22,182.162,0,1,0.610,0.830,-4.105,0.0571,0.03300,0.000000,0.1180,0.728,4,Panic! At The Disco


In [39]:
playlist1.to_csv("Playlist1.csv")
playlist2.to_csv("Playlist2.csv")

playlists.to_csv("Combined.csv")

artist1.to_csv("Artists1.csv")
artist2.to_csv("Artists2.csv")

collabs.to_csv("All Collaborations.csv", index=False)

all_artists.to_csv("All Artists.csv", index=False)

all_genres.to_csv("Genres.csv", index=False)

songs_common_genres.to_csv("Common Genre Songs.csv", index=False)
songs_common_artists.to_csv("Common Artist Songs.csv", index=False)
new_playlist.to_csv("New Playlist.csv", index=False)